## Importacao de Bibliotecas

In [47]:
import pandas as pd
import pandas_profiling as pp
import numpy as np
import seaborn as sns
import plotly.express as px

from sklearn import datasets
from matplotlib import pyplot as plt


sns.set()
# ignorando os warnings
import warnings
warnings.filterwarnings('ignore')
# plotando o matplotl
%matplotlib inline
plt.rcParams["figure.figsize"] = (5,5)
pd.set_option('display.max_columns', None)

## Importando Dataset de Amostra de Clientes

In [31]:
#csv = r'C:\Users\mauri.leite\Documents\GitHub\DH\Desafio03\Telecom - AMOSTRA CLIENTES.csv'
csv = r'/Users/mauriguedes/Documents/GitHub/DH/Desafio03/Telecom - AMOSTRA CLIENTES.csv'
dados = pd.read_csv( csv, encoding = 'latin1', 
                 engine='python', 
                 delimiter=';', 
                 decimal=",")

# Explorando os dados

In [32]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
customerID          7043 non-null object
gender              7043 non-null object
SeniorCitizen       7043 non-null int64
Partner             7043 non-null object
Dependents          7043 non-null object
tenure              7043 non-null int64
PhoneService        7043 non-null object
MultipleLines       7043 non-null object
InternetService     7043 non-null object
OnlineSecurity      7043 non-null object
OnlineBackup        7043 non-null object
DeviceProtection    7043 non-null object
TechSupport         7043 non-null object
StreamingTV         7043 non-null object
StreamingMovies     7043 non-null object
Contract            7043 non-null object
PaperlessBilling    7043 non-null object
PaymentMethod       7043 non-null object
MonthlyCharges      7043 non-null object
TotalCharges        7043 non-null object
Churn               7043 non-null object
dtypes: int64(2), object(19)
memor

In [33]:
dados.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,"R$ 29,85","R$ 29,85",No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,"R$ 56,95","R$ 1.889,50",No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,"R$ 53,85","R$ 108,15",Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),"R$ 42,30","R$ 1.840,75",No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,"R$ 70,70","R$ 151,65",Yes


# Transformação dos dados "boleanos"

#### Transformando os dados monetários em float

In [34]:
dados['MonthlyCharges'] = dados['MonthlyCharges'].replace('[R$]', '', regex=True)
dados['MonthlyCharges'] = dados['MonthlyCharges'].replace('[,]', '.', regex=True)
dados['MonthlyCharges'] = dados['MonthlyCharges'].astype(float)

dados['TotalCharges'] = dados['TotalCharges'].replace('[R$]', '', regex=True)
dados['TotalCharges'] = dados['TotalCharges'].replace('[.]', '', regex=True)
dados['TotalCharges'] = dados['TotalCharges'].replace('[,]', '.', regex=True)
dados['TotalCharges'] = dados['TotalCharges'].str.strip()
#dados['TotalCharges'] = dados['TotalCharges'].astype(float)

In [35]:
dados.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


#### Inserindo valores zerado onde tinhamos dados em branco
Ao tentar realizar a conversão do campo em float deu um erro e foi verificado a presença de dados em Branco
Primeiramente vamos colocar como 0 e depois se necessário analisar se será necessário realizar algum outro ajuste

In [36]:
dados['TotalCharges'].loc[dados[dados['TotalCharges'] == ''].index] = 0
dados['TotalCharges'] = dados['TotalCharges'].astype(float)

### Transformação do Dado Yes/No em Bolean 

- Alguns dados que possuem o 3 valor em sua categorização (ex.: "MultipleLines" [Yes, No, No phone service]), porem vejo que é uma variável dependete "PhoneService", portanto para o 3 valor iremos usar 0.

In [37]:
dados['Partner'].loc[dados[dados['Partner'] == 'Yes'].index] = 1
dados['Partner'].loc[dados[dados['Partner'] == 'No'].index] = 0
dados['Partner'] = dados['Partner'].astype(int)

dados['Dependents'].loc[dados[dados['Dependents'] == 'Yes'].index] = 1
dados['Dependents'].loc[dados[dados['Dependents'] == 'No'].index] = 0
dados['Dependents'] = dados['Dependents'].astype(int)

dados['PhoneService'].loc[dados[dados['PhoneService'] == 'Yes'].index] = 1
dados['PhoneService'].loc[dados[dados['PhoneService'] == 'No'].index] = 0
dados['PhoneService'] = dados['PhoneService'].astype(int)

dados['MultipleLines'].loc[dados[dados['MultipleLines'] == 'Yes'].index] = 1
dados['MultipleLines'].loc[dados[dados['MultipleLines'] == 'No'].index] = 0
dados['MultipleLines'].loc[dados[dados['MultipleLines'] == 'No phone service'].index] = 0
dados['MultipleLines'] = dados['MultipleLines'].astype(int)

dados['OnlineSecurity'].loc[dados[dados['OnlineSecurity'] == 'Yes'].index] = 1
dados['OnlineSecurity'].loc[dados[dados['OnlineSecurity'] == 'No'].index] = 0
dados['OnlineSecurity'].loc[dados[dados['OnlineSecurity'] == 'No internet service'].index] = 0
dados['OnlineSecurity'] = dados['OnlineSecurity'].astype(int)

dados['OnlineBackup'].loc[dados[dados['OnlineBackup'] == 'Yes'].index] = 1
dados['OnlineBackup'].loc[dados[dados['OnlineBackup'] == 'No'].index] = 0
dados['OnlineBackup'].loc[dados[dados['OnlineBackup'] == 'No internet service'].index] = 0
dados['OnlineBackup'] = dados['OnlineBackup'].astype(int)

dados['DeviceProtection'].loc[dados[dados['DeviceProtection'] == 'Yes'].index] = 1
dados['DeviceProtection'].loc[dados[dados['DeviceProtection'] == 'No'].index] = 0
dados['DeviceProtection'].loc[dados[dados['DeviceProtection'] == 'No internet service'].index] = 0
dados['DeviceProtection'] = dados['DeviceProtection'].astype(int)

dados['TechSupport'].loc[dados[dados['TechSupport'] == 'Yes'].index] = 1
dados['TechSupport'].loc[dados[dados['TechSupport'] == 'No'].index] = 0
dados['TechSupport'].loc[dados[dados['TechSupport'] == 'No internet service'].index] = 0
dados['TechSupport'] = dados['TechSupport'].astype(int)

dados['StreamingTV'].loc[dados[dados['StreamingTV'] == 'Yes'].index] = 1
dados['StreamingTV'].loc[dados[dados['StreamingTV'] == 'No'].index] = 0
dados['StreamingTV'].loc[dados[dados['StreamingTV'] == 'No internet service'].index] = 0
dados['StreamingTV'] = dados['StreamingTV'].astype(int)

dados['StreamingMovies'].loc[dados[dados['StreamingMovies'] == 'Yes'].index] = 1
dados['StreamingMovies'].loc[dados[dados['StreamingMovies'] == 'No'].index] = 0
dados['StreamingMovies'].loc[dados[dados['StreamingMovies'] == 'No internet service'].index] = 0
dados['StreamingMovies'] = dados['StreamingMovies'].astype(int)

dados['PaperlessBilling'].loc[dados[dados['PaperlessBilling'] == 'Yes'].index] = 1
dados['PaperlessBilling'].loc[dados[dados['PaperlessBilling'] == 'No'].index] = 0

dados['PaperlessBilling'] = dados['PaperlessBilling'].astype(int)

dados['Churn'].loc[dados[dados['Churn'] == 'Yes'].index] = 1
dados['Churn'].loc[dados[dados['Churn'] == 'No'].index] = 0
dados['Churn'] = dados['Churn'].astype(int)

### Criação de categorias de valores

- Categorizar as colunas de valores com base nos quartis (0-24; 25-49; 50-74; Above_75) 
- Criar dummies?

In [38]:
dados.describe().T

,count,mean,std,min,25%,50%,75%,max
SeniorCitizen,7043.0,0.162147,0.368612,0.00,0.00,0.00,0.00,1.00
Partner,7043.0,0.483033,0.499748,0.00,0.00,0.00,1.00,1.00
Dependents,7043.0,0.299588,0.458110,0.00,0.00,0.00,1.00,1.00
tenure,7043.0,32.371149,24.559481,0.00,9.00,29.00,55.00,72.00
PhoneService,7043.0,0.903166,0.295752,0.00,1.00,1.00,1.00,1.00
MultipleLines,7043.0,0.421837,0.493888,0.00,0.00,0.00,1.00,1.00
OnlineSecurity,7043.0,0.286668,0.452237,0.00,0.00,0.00,1.00,1.00
OnlineBackup,7043.0,0.344881,0.475363,0.00,0.00,0.00,1.00,1.00
DeviceProtection,7043.0,0.343888,0.475038,0.00,0.00,0.00,1.00,1.00
TechSupport,7043.0,0.290217,0.453895,0.00,0.00,0.00,1.00,1.00


In [39]:
##tenure
dados['Cat_tenure'] = np.where( dados['tenure']< 9.00 , '0-24','')
dados['Cat_tenure'] = np.where((dados['tenure']>=9.00)  & (dados['tenure']<29.00) , '25-49',dados['Cat_tenure'])
dados['Cat_tenure'] = np.where((dados['tenure']>=29.00) & (dados['tenure']<55.00) , '50-74',dados['Cat_tenure'])
dados['Cat_tenure'] = np.where( dados['tenure']>=55.00 , 'Above-75',dados['Cat_tenure'])

##MonthlyCharges
dados['Cat_MonthlyCharges'] = np.where(dados['MonthlyCharges']< 35.50, '0-24','')
dados['Cat_MonthlyCharges'] = np.where((dados['MonthlyCharges']>=35.50) & (dados['MonthlyCharges']<70.35) , '25-49',dados['Cat_MonthlyCharges'])
dados['Cat_MonthlyCharges'] = np.where((dados['MonthlyCharges']>=70.35) & (dados['MonthlyCharges']<89.85) , '50-74',dados['Cat_MonthlyCharges'])
dados['Cat_MonthlyCharges'] = np.where(dados['MonthlyCharges']>=89.85, 'Above-75',dados['Cat_MonthlyCharges'])

##TotalCharges
dados['Cat_TotalCharges'] = np.where(dados['TotalCharges']< 398.55, '0-24','')
dados['Cat_TotalCharges'] = np.where((dados['TotalCharges']>=398.55) & (dados['TotalCharges']<1394.55) , '25-49',dados['Cat_TotalCharges'])
dados['Cat_TotalCharges'] = np.where((dados['TotalCharges']>=1394.55) & (dados['TotalCharges']<3786.60) , '50-74',dados['Cat_TotalCharges'])
dados['Cat_TotalCharges'] = np.where(dados['TotalCharges']>=3786.60, 'Above-75',dados['Cat_TotalCharges'])


dados.head()


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,Cat_tenure,Cat_MonthlyCharges,Cat_TotalCharges
0,7590-VHVEG,Female,0,1,0,1,0,0,DSL,0,1,0,0,0,0,Month-to-month,1,Electronic check,29.85,29.85,0,0-24,0-24,0-24
1,5575-GNVDE,Male,0,0,0,34,1,0,DSL,1,0,1,0,0,0,One year,0,Mailed check,56.95,1889.50,0,50-74,25-49,50-74
2,3668-QPYBK,Male,0,0,0,2,1,0,DSL,1,1,0,0,0,0,Month-to-month,1,Mailed check,53.85,108.15,1,0-24,25-49,0-24
3,7795-CFOCW,Male,0,0,0,45,0,0,DSL,1,0,1,1,0,0,One year,0,Bank transfer (automatic),42.30,1840.75,0,50-74,25-49,50-74
4,9237-HQITU,Female,0,0,0,2,1,0,Fiber optic,0,0,0,0,0,0,Month-to-month,1,Electronic check,70.70,151.65,1,0-24,50-74,0-24


### Criando os dummies das categorias criadas acima

In [40]:
dados = pd.concat([dados, pd.get_dummies(dados['Cat_tenure'].astype(object), prefix='Cat_tenure', drop_first=True)], axis=1)
dados.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,Cat_tenure,Cat_MonthlyCharges,Cat_TotalCharges,Cat_tenure_25-49,Cat_tenure_50-74,Cat_tenure_Above-75
0,7590-VHVEG,Female,0,1,0,1,0,0,DSL,0,1,0,0,0,0,Month-to-month,1,Electronic check,29.85,29.85,0,0-24,0-24,0-24,0,0,0
1,5575-GNVDE,Male,0,0,0,34,1,0,DSL,1,0,1,0,0,0,One year,0,Mailed check,56.95,1889.50,0,50-74,25-49,50-74,0,1,0
2,3668-QPYBK,Male,0,0,0,2,1,0,DSL,1,1,0,0,0,0,Month-to-month,1,Mailed check,53.85,108.15,1,0-24,25-49,0-24,0,0,0
3,7795-CFOCW,Male,0,0,0,45,0,0,DSL,1,0,1,1,0,0,One year,0,Bank transfer (automatic),42.30,1840.75,0,50-74,25-49,50-74,0,1,0
4,9237-HQITU,Female,0,0,0,2,1,0,Fiber optic,0,0,0,0,0,0,Month-to-month,1,Electronic check,70.70,151.65,1,0-24,50-74,0-24,0,0,0


In [71]:
#fig, axs = plt.subplots(2)
fig = px.histogram(dados, x="Cat_tenure", color="Churn", width=800, height=400)
fig2 = px.histogram(dados, x="Cat_MonthlyCharges", color="Churn", width=800, height=400)
fig3 = px.histogram(dados, x="Cat_TotalCharges", color="Churn", width=800, height=400)


fig.show()
fig2.show()
fig3.show()

In [43]:
#sns.pairplot(dados,hue='Churn')
#sns.pairplot(dados)

### Verificar através dos serviços comprados se existe algum centroide padrao
Tentar aplicar K-neas afim de determinar grandes centros de produtos


In [44]:
# verificando o tamanho das classes por groupby
#dados.groupby('Churn').InternetService.count()